In [45]:
import os
from glob import glob
import matplotlib.pyplot as plt
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
import seaborn as sns
import zlib
import itertools
import sklearn
import itertools
import scipy
import skimage
from skimage.transform import resize
import csv
from tqdm import tqdm
from sklearn import model_selection
from sklearn.model_selection import train_test_split,learning_curve,KFold,cross_val_score,StratifiedKFold
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
import keras
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda,MaxPool2D, BatchNormalization
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils import class_weight
from keras.models import Sequential, model_from_json
from keras.layers import Activation,Dense, Dropout, Flatten, Conv2D,MaxPool2D,MaxPooling2D,AvgPool2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
#from keras.applications.mobilenet import MobileNet
#from sklearn.metrics import roc_auc_score
#from sklearn.metrics import roc_curve
#from sklearn.metrics import auc
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [46]:
train_dir = "../input/kermany2018/OCT2017 /train"
test_dir = "../input/kermany2018/OCT2017 /test"

In [47]:
for dirpath, dirname, filenames in os.walk("../input/kermany2018/OCT2017 "):
    print(f"There are {len(dirname)} directories and there are {len(filenames)} images in {dirpath}")

There are 3 directories and there are 0 images in ../input/kermany2018/OCT2017 
There are 4 directories and there are 0 images in ../input/kermany2018/OCT2017 /val
There are 0 directories and there are 8 images in ../input/kermany2018/OCT2017 /val/DRUSEN
There are 0 directories and there are 8 images in ../input/kermany2018/OCT2017 /val/CNV
There are 0 directories and there are 8 images in ../input/kermany2018/OCT2017 /val/NORMAL
There are 0 directories and there are 8 images in ../input/kermany2018/OCT2017 /val/DME
There are 4 directories and there are 0 images in ../input/kermany2018/OCT2017 /test
There are 0 directories and there are 242 images in ../input/kermany2018/OCT2017 /test/DRUSEN
There are 0 directories and there are 242 images in ../input/kermany2018/OCT2017 /test/CNV
There are 0 directories and there are 242 images in ../input/kermany2018/OCT2017 /test/NORMAL
There are 0 directories and there are 242 images in ../input/kermany2018/OCT2017 /test/DME
There are 4 directories

In [48]:
!mkdir "dataset"
!mkdir "dataset/train"
!mkdir "dataset/test"
!mkdir "dataset/train/CNV"
!mkdir "dataset/train/DRUSEN"
!mkdir "dataset/train/DME"
!mkdir "dataset/train/NORMAL"
!mkdir "dataset/test/CNV"
!mkdir "dataset/test/DRUSEN"
!mkdir "dataset/test/DME"
!mkdir "dataset/test/NORMAL"

mkdir: cannot create directory ‘dataset’: File exists
mkdir: cannot create directory ‘dataset/train’: File exists
mkdir: cannot create directory ‘dataset/test’: File exists
mkdir: cannot create directory ‘dataset/train/CNV’: File exists
mkdir: cannot create directory ‘dataset/train/DRUSEN’: File exists
mkdir: cannot create directory ‘dataset/train/DME’: File exists
mkdir: cannot create directory ‘dataset/train/NORMAL’: File exists
mkdir: cannot create directory ‘dataset/test/CNV’: File exists
mkdir: cannot create directory ‘dataset/test/DRUSEN’: File exists
mkdir: cannot create directory ‘dataset/test/DME’: File exists
mkdir: cannot create directory ‘dataset/test/NORMAL’: File exists


In [49]:
import shutil
for folder in os.listdir(train_dir):
    for file in os.listdir(train_dir + '/' + folder):
        if folder == "CNV":
            if len(os.listdir("dataset/train/CNV")) < 7000:
                shutil.copy(train_dir + '/' + folder + '/' + file, "dataset/train/CNV")
            else:
                break
        elif folder == "DRUSEN":
            if len(os.listdir("dataset/train/DRUSEN")) < 7000:
                shutil.copy(train_dir + '/' + folder + '/' + file, "dataset/train/DRUSEN")
            else:
                break
        elif folder == "DME":
            if len(os.listdir("dataset/train/DME")) < 7000:
                shutil.copy(train_dir + '/' + folder + '/' + file, "dataset/train/DME")
            else:
                break
        elif folder == "NORMAL":
            if len(os.listdir("dataset/train/NORMAL")) < 7000:
                shutil.copy(train_dir + '/' + folder + '/' + file, "dataset/train/NORMAL")
            else:
                break

In [50]:
for dirpath, dirname, filenames in os.walk("dataset"):
    print(f"There are {len(dirname)} directories and there are {len(filenames)} images in {dirpath}")

There are 2 directories and there are 0 images in dataset
There are 4 directories and there are 0 images in dataset/train
There are 0 directories and there are 7000 images in dataset/train/NORMAL
There are 0 directories and there are 7000 images in dataset/train/CNV
There are 0 directories and there are 7000 images in dataset/train/DRUSEN
There are 0 directories and there are 7000 images in dataset/train/DME
There are 4 directories and there are 0 images in dataset/test
There are 0 directories and there are 242 images in dataset/test/NORMAL
There are 0 directories and there are 242 images in dataset/test/CNV
There are 0 directories and there are 242 images in dataset/test/DRUSEN
There are 0 directories and there are 242 images in dataset/test/DME


In [51]:
for folder in os.listdir(test_dir):
    for file in os.listdir(test_dir + '/' + folder):
        if folder == "CNV":
                shutil.copy(test_dir + '/' + folder + '/' + file, "dataset/test/CNV")
        elif folder == "DRUSEN":
                shutil.copy(test_dir + '/' + folder + '/' + file, "dataset/test/DRUSEN")
        elif folder == "DME":
                shutil.copy(test_dir + '/' + folder + '/' + file, "dataset/test/DME")
        elif folder == "NORMAL":
                shutil.copy(test_dir + '/' + folder + '/' + file, "dataset/test/NORMAL")

In [52]:
for dirpath, dirname, filenames in os.walk("dataset"):
    print(f"There are {len(dirname)} directories and there are {len(filenames)} images in {dirpath}")

There are 2 directories and there are 0 images in dataset
There are 4 directories and there are 0 images in dataset/train
There are 0 directories and there are 7000 images in dataset/train/NORMAL
There are 0 directories and there are 7000 images in dataset/train/CNV
There are 0 directories and there are 7000 images in dataset/train/DRUSEN
There are 0 directories and there are 7000 images in dataset/train/DME
There are 4 directories and there are 0 images in dataset/test
There are 0 directories and there are 242 images in dataset/test/NORMAL
There are 0 directories and there are 242 images in dataset/test/CNV
There are 0 directories and there are 242 images in dataset/test/DRUSEN
There are 0 directories and there are 242 images in dataset/test/DME


In [53]:
import tensorflow as tf
train_data = tf.keras.preprocessing.image_dataset_from_directory("dataset/train",
                                                                image_size = (224, 224),
                                                                label_mode = "categorical",
                                                                shuffle = True)
test_data = tf.keras.preprocessing.image_dataset_from_directory("dataset/test",
                                                                image_size = (224, 224),
                                                                label_mode = "categorical",
                                                                shuffle = True)
train_data, test_data

Found 28000 files belonging to 4 classes.
Found 968 files belonging to 4 classes.


(<BatchDataset shapes: ((None, 224, 224, 3), (None, 4)), types: (tf.float32, tf.float32)>,
 <BatchDataset shapes: ((None, 224, 224, 3), (None, 4)), types: (tf.float32, tf.float32)>)

In [54]:
train_dataset = train_data.prefetch(tf.data.AUTOTUNE)
test_dataset = test_data.prefetch(tf.data.AUTOTUNE)
train_dataset, test_dataset

(<PrefetchDataset shapes: ((None, 224, 224, 3), (None, 4)), types: (tf.float32, tf.float32)>,
 <PrefetchDataset shapes: ((None, 224, 224, 3), (None, 4)), types: (tf.float32, tf.float32)>)

In [55]:
#import tensorflow_hub as hub
#feature_extraction_layer = hub.KerasLayer("https://tfhub.dev/emilutz/vgg19-block5-conv2-unpooling-decoder/1",
#                                         trainable = False,
#                                         input_shape = (224, 224, 3))

In [56]:
tf.random.set_seed(42)

In [57]:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3),  activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3),  activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3),  activation="relu"))
#model.add(Conv2D(filters=128, kernel_size=(3,3),  activation="relu"))
#model.add(Conv2D(filters=128, kernel_size=(3,3),  activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3),  activation="relu"))
#model.add(Conv2D(filters=256, kernel_size=(3,3),  activation="relu"))
#model.add(Conv2D(filters=256, kernel_size=(3,3),  activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3),  activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3),  activation="relu"))
#model.add(Conv2D(filters=512, kernel_size=(3,3),  activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(tf.keras.layers.GlobalAveragePooling2D())
#model.add(Dense(units=64,activation="relu"))
#model.add(Dense(units=32,activation="relu"))
model.add(Dense(units=4, activation="softmax"))

In [58]:
model.compile(loss = "categorical_crossentropy", optimizer = "Adam", metrics = ["accuracy"])

model_history = model.fit(train_dataset,
                         epochs = 50,
                         validation_data = test_dataset)

Epoch 1/50
875/875 [==============================] - 111s 125ms/step - loss: 1.1104 - accuracy: 0.6056 - val_loss: 0.3742 - val_accuracy: 0.8709
Epoch 2/50
875/875 [==============================] - 110s 125ms/step - loss: 0.4759 - accuracy: 0.8275 - val_loss: 0.0818 - val_accuracy: 0.9793
Epoch 3/50
875/875 [==============================] - 110s 125ms/step - loss: 0.3684 - accuracy: 0.8747 - val_loss: 0.1223 - val_accuracy: 0.9628
Epoch 4/50
875/875 [==============================] - 110s 125ms/step - loss: 0.3075 - accuracy: 0.8964 - val_loss: 0.1124 - val_accuracy: 0.9700
Epoch 5/50
875/875 [==============================] - 110s 125ms/step - loss: 0.2736 - accuracy: 0.9075 - val_loss: 0.0593 - val_accuracy: 0.9866
Epoch 6/50
875/875 [==============================] - 110s 125ms/step - loss: 0.2428 - accuracy: 0.9190 - val_loss: 0.0518 - val_accuracy: 0.9876
Epoch 7/50
875/875 [==============================] - 110s 125ms/step - loss: 0.2235 - accuracy: 0.9235 - val_loss: 0.0679 -

KeyboardInterrupt: 

In [ ]:
model.summary()

In [ ]:
history = model.history.history
def plot_metrics(history):
  
    train_loss = history['loss']
    val_loss = history['val_loss']
    train_acc = history['accuracy']
    val_acc = history['val_accuracy']
    
    # Loss
    plt.figure()
    plt.plot(train_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.title('Loss')
    plt.ylabel('Loss')
    plt.xlabel('epochs')
    plt.legend()
    plt.show()
    
    # Accuracy
    plt.figure()
    plt.plot(train_acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.title('Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('epochs')
    plt.legend()
    plt.show()

In [ ]:
plot_metrics(history)

In [ ]:
# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224):
  #"""Reads an image from filename, turns it into a tensor and reshapes it to (img_shape, img_shape, colour_channel)."""
  # Read in target file (an image)
  img = tf.io.read_file(filename)

  # Decode the read file into a tensor & ensure 3 colour channels 
  # (our model is trained on images with 3 colour channels and sometimes images have 4 colour channels)
  img = tf.image.decode_image(img, channels=3)

  # Resize the image (to the same size our model was trained on)
  img = tf.image.resize(img, size = [img_shape, img_shape])
  # Rescale the image (get all values between 0 and 1)
  img = img/255.
  return img

In [ ]:
class_names = train_data.class_names
print(class_names)

In [ ]:
def pred_and_plot(model, filename, class_names):
  #"""Imports an image located at filename, makes a prediction on it with a trained model and plots the image with the predicted class as the title."""
  # Import the target image and preprocess it
  img = load_and_prep_image(filename)
  plt.imshow(img)
  plt.show()
  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))
  print(pred, tf.argmax(pred, axis = 1))
  # Get the predicted class
  pred_class = class_names[tf.argmax(pred, axis = 1).numpy()[0]]
  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);

In [ ]:
file_name="../input/kermany2018/OCT2017 /val/DRUSEN/DRUSEN-9837663-1.jpeg"
pred_and_plot(model,file_name,class_names)

In [ ]:
pred1=model.predict(test_data,batch_size=50)

In [ ]:
prd=[]
for i in range(len(pred1)):
    prd.append(np.argmax(pred1.round()[i]))
conf=pd.DataFrame(confusion_matrix([[0,1,2,3]],prd))

In [ ]:
print('\nConfusion Matrix ')
conf

In [ ]:
print('\nConfusion Matrix Graph')
fig=plt.figure(figsize=(5,5))
sns.heatmap(conf,annot=True,cmap='viridis',cbar=False,fmt='.5g')
plt.show()